# Le PIB est-il un indicateur robuste de l'accès à l'alimentation ?

In [11]:
import requests
import pandas as pd
import numpy as np

## Partie 1 : Collecte des données

### Récupération des données de PIB à partir de la base de données de la banque mondiale

In [12]:
url = "https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.KD?format=json&per_page=20000"


response = requests.get(url)

# Extraction du JSON de la réponse
PIB_hab = response.json()  # ici c’est correct

# Vérification des données
print(type(PIB_hab),len(PIB_hab))
print(PIB_hab[0]) # Métadonnées
print(PIB_hab[1][:5]) # Données de PIB par pays


<class 'list'> 2
{'page': 1, 'pages': 1, 'per_page': 20000, 'total': 17290, 'sourceid': '2', 'lastupdated': '2025-10-07'}
[{'indicator': {'id': 'NY.GDP.PCAP.KD', 'value': 'GDP per capita (constant 2015 US$)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2024', 'value': 1416.25036869408, 'unit': '', 'obs_status': '', 'decimal': 1}, {'indicator': {'id': 'NY.GDP.PCAP.KD', 'value': 'GDP per capita (constant 2015 US$)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2023', 'value': 1412.62538438423, 'unit': '', 'obs_status': '', 'decimal': 1}, {'indicator': {'id': 'NY.GDP.PCAP.KD', 'value': 'GDP per capita (constant 2015 US$)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2022', 'value': 1421.79716897878, 'unit': '', 'obs_status': '', 'decimal': 1}, {'indicator': {'id': 'NY.GDP.PCAP.KD', 'value': 'GDP per capita (constant 2015 

#Retraitement des données


Suppression des colonnes inutiles

In [13]:
donnees = PIB_hab[1]  # On ne garde pas les métadonnées

# On transforme le JSON en DataFrame et on ne garde que le PIB/hab pour chaque années avec le nom et le code du pays
liste_simplifiee = []
for item in donnees:
    nom_pays = item['country']['value']  # nom complet du pays
    annee = int(item['date'])
    pib = item['value']
    liste_simplifiee.append({'nom_pays': nom_pays, 'année': annee, 'PIB_par_habitant': pib})

PIB_hab_simple = pd.DataFrame(liste_simplifiee)

# Filtrer uniquement les années de 1960 à 2022. 
PIB_hab_simple = PIB_hab_simple[(PIB_hab_simple['année'] >= 1960) & (PIB_hab_simple['année'] <= 2022)]

# Afficher les 5 premières lignes
print(PIB_hab_simple.head(200))

                        nom_pays  année  PIB_par_habitant
2    Africa Eastern and Southern   2022       1421.797169
3    Africa Eastern and Southern   2021       1409.040699
4    Africa Eastern and Southern   2020       1383.724119
5    Africa Eastern and Southern   2019       1463.437891
6    Africa Eastern and Southern   2018       1471.438317
..                           ...    ...               ...
203       Caribbean small states   2016      13879.439408
204       Caribbean small states   2015      14402.472578
205       Caribbean small states   2014      14514.365526
206       Caribbean small states   2013      14247.783112
207       Caribbean small states   2012      14138.383887

[200 rows x 3 columns]


Filtrage pour enlever les pays contenant des valeurs manquantes

In [14]:
PIB_hab_simple_sans_NA = PIB_hab_simple.dropna()
print(len(PIB_hab_simple['nom_pays'])/(2022-1960+1),len(PIB_hab_simple_sans_NA['nom_pays'])//(2022-1960+1)) # On a enlevé 48 pays ou groupe de pays de la liste

266.0 218


Restriction aux pays suivants : France, Etats-unis, Angleterre, Russie, Chine, Japon, Brésil, Afrique du Sud

In [15]:
# Liste des pays à conserver
pays_a_conserver = [
    'France', 'United States', 'United Kingdom', 
    'Russia', 'China', 'Japan', 'Brazil', 'South Africa'
]

# Filtrer le DataFrame
PIB_hab_filtré = PIB_hab_simple_sans_NA[PIB_hab_simple_sans_NA['nom_pays'].isin(pays_a_conserver)]

# Vérifier le résultat
print(PIB_hab_filtré['nom_pays'].unique())
print(PIB_hab_filtré.head(126))

['Brazil' 'China' 'France' 'Japan' 'South Africa' 'United Kingdom'
 'United States']
     nom_pays  année  PIB_par_habitant
4877   Brazil   2022       9032.084816
4878   Brazil   2021       8799.229458
4879   Brazil   2020       8435.011433
4880   Brazil   2019       8771.440513
4881   Brazil   2018       8722.336303
...       ...    ...               ...
5910    China   1964        206.399719
5911    China   1963        178.749039
5912    China   1962        166.089290
5913    China   1961        177.353934
5914    China   1960        241.388656

[126 rows x 3 columns]
